In [1]:
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as psql
from datetime import timedelta

import warnings
warnings.filterwarnings('ignore')

Connect to the database and fetch the person_visit_death_with_concepts table

In [2]:
def fetch_data(dbname, user, password, table):
    connection = pg.connect('host=localhost dbname='+dbname+' user='+user+' password=\''+password+'\'')
    df = pd.read_sql_query('SELECT * FROM '+table, con=connection)
    connection.close()
    return df

In [3]:
dbname = 'commondatamodel'
user = 'romirmoza'
password = ''
table = 'person_visit_death_with_concepts'
df = fetch_data(dbname, user, password, table)

Convert dates to the correct datatype

In [4]:
df.columns

Index(['year_of_birth', 'ethnicity_concept_id', 'person_id', 'month_of_birth',
       'day_of_birth', 'race_concept_id', 'gender_concept_id',
       'birth_datetime', 'visit_start_date', 'preceding_visit_occurrence_id',
       'visit_occurrence_id', 'visit_end_date', 'visit_concept_id',
       'visit_type_concept_id', 'discharge_to_concept_id', 'race_concept_name',
       'visit_concept_name', 'death_date', 'death_datetime',
       'death_type_concept_id'],
      dtype='object')

In [5]:
df[['visit_start_date','visit_end_date', 'death_date']] = \
df[['visit_start_date','visit_end_date', 'death_date']].apply(pd.to_datetime, format='%Y-%m-%d')

In [6]:
len(df)

1000

Add visit_duration columns

In [7]:
df['visit_duration'] = df['visit_end_date'] - df['visit_start_date']

In [8]:
df.head()

,year_of_birth,ethnicity_concept_id,person_id,month_of_birth,day_of_birth,race_concept_id,gender_concept_id,birth_datetime,visit_start_date,preceding_visit_occurrence_id,...,visit_end_date,visit_concept_id,visit_type_concept_id,discharge_to_concept_id,race_concept_name,visit_concept_name,death_date,death_datetime,death_type_concept_id,visit_duration
0,1923,NaN,0,5,1,8552.0,8507,NaT,2009-03-31,None,...,2009-03-31,0,44818517,NaN,Unknown,None,NaT,NaT,NaN,0 days
1,1923,NaN,0,5,1,8552.0,8507,NaT,2008-04-11,None,...,2008-04-11,0,44818517,44814705.0,Unknown,None,NaT,NaT,NaN,0 days
2,1923,NaN,0,5,1,8552.0,8507,NaT,2009-03-14,None,...,NaT,0,44818517,44814705.0,Unknown,None,NaT,NaT,NaN,NaT
3,1923,NaN,0,5,1,8552.0,8507,NaT,2010-03-12,None,...,NaT,9202,44818517,NaN,Unknown,Outpatient Visit,NaT,NaT,NaN,NaT
4,1923,NaN,0,5,1,8552.0,8507,NaT,2008-09-15,None,...,2008-09-15,0,44818517,44814705.0,Unknown,None,NaT,NaT,NaN,0 days


In [9]:
df['visit_end_date'] = df['visit_end_date'].fillna(df['visit_start_date']) 

In [10]:
df['death_date'] = df['death_date'].fillna(pd.Timestamp.max)

In [11]:
df[df['death_date'] != pd.Timestamp.max].death_date

358   2008-05-19
Name: death_date, dtype: datetime64[ns]

In [12]:
df.head()

,year_of_birth,ethnicity_concept_id,person_id,month_of_birth,day_of_birth,race_concept_id,gender_concept_id,birth_datetime,visit_start_date,preceding_visit_occurrence_id,...,visit_end_date,visit_concept_id,visit_type_concept_id,discharge_to_concept_id,race_concept_name,visit_concept_name,death_date,death_datetime,death_type_concept_id,visit_duration
0,1923,NaN,0,5,1,8552.0,8507,NaT,2009-03-31,None,...,2009-03-31,0,44818517,NaN,Unknown,None,2262-04-11 23:47:16.854775807,NaT,NaN,0 days
1,1923,NaN,0,5,1,8552.0,8507,NaT,2008-04-11,None,...,2008-04-11,0,44818517,44814705.0,Unknown,None,2262-04-11 23:47:16.854775807,NaT,NaN,0 days
2,1923,NaN,0,5,1,8552.0,8507,NaT,2009-03-14,None,...,2009-03-14,0,44818517,44814705.0,Unknown,None,2262-04-11 23:47:16.854775807,NaT,NaN,NaT
3,1923,NaN,0,5,1,8552.0,8507,NaT,2010-03-12,None,...,2010-03-12,9202,44818517,NaN,Unknown,Outpatient Visit,2262-04-11 23:47:16.854775807,NaT,NaN,NaT
4,1923,NaN,0,5,1,8552.0,8507,NaT,2008-09-15,None,...,2008-09-15,0,44818517,44814705.0,Unknown,None,2262-04-11 23:47:16.854775807,NaT,NaN,0 days


In [13]:
max_visit_start_date =  df['visit_start_date'].max()
min_visit_start_date =  df['visit_start_date'].min()
print(max_visit_start_date)
print(min_visit_start_date)

2010-05-25 00:00:00
2008-01-06 00:00:00


In [14]:
def check_death_flag(x, window_size):
    if x.death_date - x.visit_start_date < window_size and x.death_date - x.visit_start_date >= timedelta(days = 0):
        return 1
    return 0

Generate windows of training data with window_id being the identifier. Every row has a death_in_next_window field that informs us whether the person dies in the next window.

In [15]:
def visit_types_count(x):
    return pd.Series(dict(
        inpatient_visit_count  = (x.visit_concept_name == 'Inpatient Visit').sum(),
        outpatient_visit_count = (x.visit_concept_name == 'Outpatient Visit').sum(),
        er_visit_count         = (x.visit_concept_name == 'Emergency Room Visit').sum()
        ))

In [16]:
def window_data(df, window_size, window_start, group_by_var, date_var, agg_dict, rename_dict, apply_func, calc_death=0):
    window_id = 0
    while window_start < max_visit_start_date:
        df_window = df[(df[date_var] >= window_start) & (df[date_var] < window_start + window_size)]
        if(calc_death):
            df_window['death_in_next_window'] = df_window.apply(lambda x: check_death_flag(x, window_size), axis=1)
            df_window['old'] = window_start.year - df_window.year_of_birth

        df_window[date_var] = (window_start + window_size) - df_window[date_var]
        agg_df = df_window.groupby(group_by_var).agg(agg_dict).rename(columns=rename_dict)
        apply_cols = df_window.groupby(group_by_var).apply(lambda x: apply_func(x))    
        agg_df = agg_df.join(apply_cols)
        agg_df['window_id'] = window_id
        agg_df.reset_index(drop=True)
        if not window_id:
            windowed_data = agg_df.copy()
        else:
            windowed_data = pd.concat([windowed_data, agg_df], ignore_index=True)
        window_id += 1
        window_start += window_size
    return windowed_data

In [17]:
window_size = timedelta(days = 180)
window_start = min_visit_start_date

In [18]:
agg_dict = {'person_id': 'max',
            'year_of_birth': 'max',
            'birth_datetime': 'max',
            'visit_start_date': 'min',
            'ethnicity_concept_id': 'max',
            'race_concept_id': 'max',
            'gender_concept_id': 'max',
            'race_concept_name': 'max',
            'visit_occurrence_id': 'nunique',
            'visit_concept_name': 'count',
            'visit_duration': 'sum',
            'death_in_next_window': 'max',
            'old': 'max'}

rename_dict = {'visit_occurrence_id': 'number_of_visits',
               'visit_start_date': 'days_since_latest_visit'}

group_by_var = 'person_id'
date_var = 'visit_start_date'
apply_func = visit_types_count

training_data = \
window_data(df, window_size, window_start, group_by_var, date_var, agg_dict, rename_dict, apply_func, 1)

In [19]:
training_data.head()

,person_id,year_of_birth,birth_datetime,days_since_latest_visit,ethnicity_concept_id,race_concept_id,gender_concept_id,race_concept_name,number_of_visits,visit_concept_name,visit_duration,death_in_next_window,old,inpatient_visit_count,outpatient_visit_count,er_visit_count,window_id
0,0,1923,NaT,4 days,NaN,8552.0,8507,Unknown,5,1,0 days,0,85,0,1,0,0
1,6,1976,NaT,142 days,NaN,8527.0,8532,White,3,0,0 days,0,32,0,0,0,0
2,10,1936,1936-07-01,123 days,38003564.0,8552.0,8532,Unknown,1,0,0 days,0,72,0,0,0,0
3,13,1942,NaT,10 days,38003564.0,8552.0,8532,Unknown,5,2,0 days,0,66,0,2,0,0
4,16,1932,NaT,81 days,NaN,8516.0,8507,Black or African American,1,0,0 days,0,76,0,0,0,0


In [20]:
training_data.dtypes

person_id                            int64
year_of_birth                        int64
birth_datetime              datetime64[ns]
days_since_latest_visit    timedelta64[ns]
ethnicity_concept_id               float64
race_concept_id                    float64
gender_concept_id                    int64
race_concept_name                   object
number_of_visits                     int64
visit_concept_name                   int64
visit_duration             timedelta64[ns]
death_in_next_window                 int64
old                                  int64
inpatient_visit_count                int64
outpatient_visit_count               int64
er_visit_count                       int64
window_id                            int64
dtype: object

In [21]:
training_data.days_since_latest_visit

0       4 days
1     142 days
2     123 days
3      10 days
4      81 days
        ...   
338    33 days
339   105 days
340    70 days
341    50 days
342   116 days
Name: days_since_latest_visit, Length: 343, dtype: timedelta64[ns]

In [22]:
training_data.head()

,person_id,year_of_birth,birth_datetime,days_since_latest_visit,ethnicity_concept_id,race_concept_id,gender_concept_id,race_concept_name,number_of_visits,visit_concept_name,visit_duration,death_in_next_window,old,inpatient_visit_count,outpatient_visit_count,er_visit_count,window_id
0,0,1923,NaT,4 days,NaN,8552.0,8507,Unknown,5,1,0 days,0,85,0,1,0,0
1,6,1976,NaT,142 days,NaN,8527.0,8532,White,3,0,0 days,0,32,0,0,0,0
2,10,1936,1936-07-01,123 days,38003564.0,8552.0,8532,Unknown,1,0,0 days,0,72,0,0,0,0
3,13,1942,NaT,10 days,38003564.0,8552.0,8532,Unknown,5,2,0 days,0,66,0,2,0,0
4,16,1932,NaT,81 days,NaN,8516.0,8507,Black or African American,1,0,0 days,0,76,0,0,0,0


In [23]:
training_data = training_data.drop(['year_of_birth', 'birth_datetime'], axis=1)

In [24]:
training_data[training_data.person_id == 2225]

,person_id,days_since_latest_visit,ethnicity_concept_id,race_concept_id,gender_concept_id,race_concept_name,number_of_visits,visit_concept_name,visit_duration,death_in_next_window,old,inpatient_visit_count,outpatient_visit_count,er_visit_count,window_id


In [25]:
training_data.window_id.unique()

array([0, 1, 2, 3, 4])

In [26]:
training_data.er_visit_count.unique()

array([0])

### Merge with condition_occurrence

In [27]:
table = 'condition_occurrence'
df = fetch_data(dbname, user, password, table)

In [28]:
df['condition_end_date'] = df['condition_end_date'] if not 'NaT' else df['condition_start_date']
df = df.drop(['condition_start_datetime',
              'condition_end_datetime',
              'stop_reason',
              'provider_id',
             'visit_detail_id',
             'condition_source_value',
             'condition_status_source_value',
             'condition_source_concept_id'], axis=1, errors='ignore')

In [29]:
df['condition_concept_id'] = df['condition_concept_id'].apply(str)
df['condition_type_concept_id'] = df['condition_type_concept_id'].apply(str)
df['condition_status_concept_id'] = df['condition_status_concept_id'].apply(str)

In [30]:
df.head()

,condition_occurrence_id,person_id,condition_concept_id,condition_start_date,condition_end_date,condition_type_concept_id,condition_status_concept_id,visit_occurrence_id
0,8901459,6,72993,2009-08-05,2009-08-05,38000230,4230359.0,979697.0
1,8891634,6,313217,2009-08-15,2009-08-15,38000230,4230359.0,2279847.0
2,9765782,6,257011,2009-01-12,2009-01-12,38000230,nan,NaN
3,13768748,6,201826,2009-05-15,2009-05-15,38000230,nan,NaN
4,6882817,6,73553,2009-01-02,2009-01-02,38000230,4230359.0,NaN


In [31]:
max_condition_start_date =  df['condition_start_date'].max()
min_condition_start_date =  df['condition_start_date'].min()
print(max_condition_start_date)
print(min_condition_start_date)

2010-05-24
2008-01-05


In [32]:
def agg_condition_concept_id(x):
    return pd.Series(dict(
        condition_concept_id_list  = ', '.join(set(x.condition_concept_id)),
        condition_type_concept_id_list  = ', '.join(set(x.condition_type_concept_id))
        ))

In [33]:
agg_dict = {'person_id': 'max',
            'condition_start_date': 'min',
            'condition_status_concept_id': 'max'}

rename_dict = {'condition_start_date': 'days_since_latest_condition'}

group_by_var = 'person_id'
date_var = 'condition_start_date'
apply_func = agg_condition_concept_id

df.condition_start_date = pd.to_datetime(df.condition_start_date, format='%Y-%m-%d')
cond_occur_data = \
window_data(df, window_size, window_start, group_by_var, date_var, agg_dict, rename_dict, apply_func)

In [34]:
cond_occur_data.head()

,person_id,days_since_latest_condition,condition_status_concept_id,condition_concept_id_list,condition_type_concept_id_list,window_id
0,6,1 days,nan,"81390, 373503, 444382, 313007, 443776, 4302043...","38000200, 38000230",0
1,7,40 days,nan,"201826, 313217, 141932, 439297",38000230,0
2,8,7 days,nan,"133810, 195195, 4188191, 134736, 313217",38000230,0
3,10,80 days,nan,"138994, 4093672",38000230,0
4,16,31 days,nan,372892,38000230,0


In [35]:
training_data = pd.merge(training_data, cond_occur_data, on=['person_id', 'window_id'], how='left')
training_data.head()

,person_id,days_since_latest_visit,ethnicity_concept_id,race_concept_id,gender_concept_id,race_concept_name,number_of_visits,visit_concept_name,visit_duration,death_in_next_window,old,inpatient_visit_count,outpatient_visit_count,er_visit_count,window_id,days_since_latest_condition,condition_status_concept_id,condition_concept_id_list,condition_type_concept_id_list
0,0,4 days,NaN,8552.0,8507,Unknown,5,1,0 days,0,85,0,1,0,0,NaT,NaN,NaN,NaN
1,6,142 days,NaN,8527.0,8532,White,3,0,0 days,0,32,0,0,0,0,1 days,nan,"81390, 373503, 444382, 313007, 443776, 4302043...","38000200, 38000230"
2,10,123 days,38003564.0,8552.0,8532,Unknown,1,0,0 days,0,72,0,0,0,0,80 days,nan,"138994, 4093672",38000230
3,13,10 days,38003564.0,8552.0,8532,Unknown,5,2,0 days,0,66,0,2,0,0,NaT,NaN,NaN,NaN
4,16,81 days,NaN,8516.0,8507,Black or African American,1,0,0 days,0,76,0,0,0,0,31 days,nan,372892,38000230


In [36]:
del cond_occur_data

### Merge with procedure_occurrence

In [37]:
table = 'procedure_occurrence'
df = fetch_data(dbname, user, password, table)

In [38]:
df.head()

,procedure_occurrence_id,person_id,procedure_concept_id,procedure_date,procedure_datetime,procedure_type_concept_id,modifier_concept_id,quantity,provider_id,visit_occurrence_id,visit_detail_id,procedure_source_value,procedure_source_concept_id,modifier_source_value
0,11177430,0,4306780,2008-05-03,2008-05-03,38000269,None,None,NaN,1435523.0,None,V7231,44828600,None
1,5077664,0,2005317,2009-08-28,2009-08-28,38000269,None,None,203540.0,462902.0,None,7862,2005317,None
2,7547623,0,2314269,2009-03-16,2009-03-16,38000269,None,None,NaN,680782.0,None,97012,2314269,None
3,462612,0,2313881,2008-07-07,2008-07-07,38000269,None,None,113568.0,5484318.0,None,93325,2313881,None
4,4592206,0,2003090,2009-03-13,2009-03-13,38000269,None,None,NaN,4011904.0,None,496,2003090,None


In [39]:
df = df.drop(['procedure_source_value',
              'procedure_source_concept_id',
              'modifier_source_value',
              'visit_detail_id',
              'provider_id',
              'quantity',
              'modifier_concept_id',
              'procedure_datetime'], axis=1, errors='ignore')

In [40]:
df['procedure_concept_id'] = df['procedure_concept_id'].apply(str)
df['procedure_type_concept_id'] = df['procedure_type_concept_id'].apply(str)

In [41]:
def agg_procedure_concept_id(x):
    return pd.Series(dict(
        procedure_concept_id_list  = ', '.join(set(x.procedure_concept_id)),
        procedure_type_concept_id_list  = ', '.join(set(x.procedure_type_concept_id))
        ))

In [42]:
agg_dict = {'person_id': 'max',
            'procedure_date': 'min'}

rename_dict = {'procedure_date': 'days_since_latest_procedure'}

group_by_var = 'person_id'
date_var = 'procedure_date'
apply_func = agg_procedure_concept_id

df.procedure_date = pd.to_datetime(df.procedure_date, format='%Y-%m-%d')
procedure_occur_data = \
window_data(df, window_size, window_start, group_by_var, date_var, agg_dict, rename_dict, apply_func)

In [43]:
training_data = pd.merge(training_data, procedure_occur_data, on=['person_id', 'window_id'], how='left')
training_data.head()

,person_id,days_since_latest_visit,ethnicity_concept_id,race_concept_id,gender_concept_id,race_concept_name,number_of_visits,visit_concept_name,visit_duration,death_in_next_window,...,outpatient_visit_count,er_visit_count,window_id,days_since_latest_condition,condition_status_concept_id,condition_concept_id_list,condition_type_concept_id_list,days_since_latest_procedure,procedure_concept_id_list,procedure_type_concept_id_list
0,0,4 days,NaN,8552.0,8507,Unknown,5,1,0 days,0,...,1,0,0,NaT,NaN,NaN,NaN,62 days,"2414392, 4048727, 4306780",38000269
1,6,142 days,NaN,8527.0,8532,White,3,0,0 days,0,...,0,0,0,1 days,nan,"81390, 373503, 444382, 313007, 443776, 4302043...","38000200, 38000230",NaT,NaN,NaN
2,10,123 days,38003564.0,8552.0,8532,Unknown,1,0,0 days,0,...,0,0,0,80 days,nan,"138994, 4093672",38000230,32 days,"2108115, 2211338, 2001200, 0",38000269
3,13,10 days,38003564.0,8552.0,8532,Unknown,5,2,0 days,0,...,2,0,0,NaT,NaN,NaN,NaN,NaT,NaN,NaN
4,16,81 days,NaN,8516.0,8507,Black or African American,1,0,0 days,0,...,0,0,0,31 days,nan,372892,38000230,2 days,"2314285, 2514408, 2002291",38000269


In [44]:
del procedure_occur_data

### Merge with drug_exposure

In [45]:
table = 'drug_exposure'
df = fetch_data(dbname, user, password, table)

In [46]:
df.head()

,drug_exposure_id,person_id,drug_concept_id,drug_exposure_start_date,drug_exposure_start_datetime,drug_exposure_end_date,drug_exposure_end_datetime,verbatim_end_date,drug_type_concept_id,stop_reason,...,sig,route_concept_id,lot_number,provider_id,visit_occurrence_id,visit_detail_id,drug_source_value,drug_source_concept_id,route_source_value,dose_unit_source_value
0,3744808,8,43560452,2010-02-18,2010-02-18 23:49:27,None,None,2010-11-21,38000175,None,...,None,None,None,NaN,3000253.0,None,54868094800,45054001,None,None
1,3866327,10,1363057,2009-09-22,2009-09-22 12:54:50,None,None,None,38000175,None,...,None,None,None,NaN,3886276.0,None,61392065460,45056800,None,None
2,1084422,10,19078924,2008-08-25,2008-08-25 10:05:57,None,None,None,38000175,None,...,None,None,None,NaN,NaN,None,64679076203,44887586,None,None
3,4796682,10,19077498,2009-03-09,2009-03-09 14:22:34,None,None,None,38000175,None,...,None,None,None,NaN,NaN,None,66267023360,45194371,None,None
4,88654,10,1545997,2009-12-19,2009-12-19 23:35:16,None,None,2014-05-21,38000175,None,...,None,None,None,NaN,NaN,None,58016025221,45072463,None,None


In [47]:
df = df.drop(['drug_exposure_start_datetime',
              'drug_exposure_end_date',
              'drug_exposure_end_datetime',
              'verbatim_end_date',
              'stop_reason',
              'lot_number',
              'provider_id',
              'visit_detail_id',
              'drug_source_value',
              'drug_source_concept_id',
              'route_source_value',
              'dose_unit_source_value',
              'route_concept_id',
              'sig',
              'refills',
              'days_supply'], axis=1, errors='ignore')

In [48]:
df['drug_concept_id'] = df['drug_concept_id'].apply(str)
df['drug_type_concept_id'] = df['drug_type_concept_id'].apply(str)

In [49]:
def agg_drug_concept_id(x):
    return pd.Series(dict(
        drug_concept_id_list  = ', '.join(set(x.drug_concept_id)),
        drug_type_concept_id_list  = ', '.join(set(x.drug_type_concept_id))
        ))

In [50]:
agg_dict = {'person_id': 'max',
            'drug_exposure_start_date': 'min',
            'quantity': 'sum'}

rename_dict = {'drug_exposure_start_date': 'days_since_latest_drug_exposure',
               'quantity': 'total_quantity_of_drugs'}

group_by_var = 'person_id'
date_var = 'drug_exposure_start_date'
apply_func = agg_drug_concept_id

df.drug_exposure_start_date = pd.to_datetime(df.drug_exposure_start_date, format='%Y-%m-%d')
drug_exposure_data = \
window_data(df, window_size, window_start, group_by_var, date_var, agg_dict, rename_dict, apply_func)

In [51]:
training_data = pd.merge(training_data, drug_exposure_data, on=['person_id', 'window_id'], how='left')
training_data.head()

,person_id,days_since_latest_visit,ethnicity_concept_id,race_concept_id,gender_concept_id,race_concept_name,number_of_visits,visit_concept_name,visit_duration,death_in_next_window,...,condition_status_concept_id,condition_concept_id_list,condition_type_concept_id_list,days_since_latest_procedure,procedure_concept_id_list,procedure_type_concept_id_list,days_since_latest_drug_exposure,total_quantity_of_drugs,drug_concept_id_list,drug_type_concept_id_list
0,0,4 days,NaN,8552.0,8507,Unknown,5,1,0 days,0,...,NaN,NaN,NaN,62 days,"2414392, 4048727, 4306780",38000269,NaT,NaN,NaN,NaN
1,6,142 days,NaN,8527.0,8532,White,3,0,0 days,0,...,nan,"81390, 373503, 444382, 313007, 443776, 4302043...","38000200, 38000230",NaT,NaN,NaN,NaT,NaN,NaN,NaN
2,10,123 days,38003564.0,8552.0,8532,Unknown,1,0,0 days,0,...,nan,"138994, 4093672",38000230,32 days,"2108115, 2211338, 2001200, 0",38000269,33 days,110.0,"798875, 19133566",38000175
3,13,10 days,38003564.0,8552.0,8532,Unknown,5,2,0 days,0,...,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN
4,16,81 days,NaN,8516.0,8507,Black or African American,1,0,0 days,0,...,nan,372892,38000230,2 days,"2314285, 2514408, 2002291",38000269,39 days,60.0,"19102504, 40006185, 1309069",38000175


In [52]:
del drug_exposure_data

### Merge with observation

In [53]:
table = 'observation'
df = fetch_data(dbname, user, password, table)

In [54]:
df.head()

,observation_id,person_id,observation_concept_id,observation_date,observation_datetime,observation_type_concept_id,value_as_number,value_as_string,value_as_concept_id,qualifier_concept_id,...,provider_id,visit_occurrence_id,visit_detail_id,observation_source_value,observation_source_concept_id,unit_source_value,qualifier_source_value,observation_event_id,obs_event_field_concept_id,value_as_datetime
0,1802194,0,440927,2009-06-01,2009-06-01 07:35:03,38000282,None,14-20,4069776.0,None,...,46319.0,NaN,None,V5863,44830822.0,None,None,None,None,None
1,583362,0,4015724,2009-09-27,2009-09-27 04:50:11,38000282,None,1-3,NaN,None,...,274541.0,NaN,None,V0481,44837741.0,None,None,None,None,None
2,1132254,0,439405,2008-05-23,2008-05-23 05:35:48,38000282,None,48,NaN,None,...,270459.0,NaN,None,7197,44829217.0,None,None,None,None,None
3,760346,0,440922,2008-12-24,2008-12-24 08:04:37,38000282,None,.2,NaN,None,...,310343.0,NaN,None,V5867,NaN,None,None,None,None,None
4,1414625,0,2614666,2010-04-26,2010-04-26 12:09:46,38000282,None,.1,4180790.0,None,...,NaN,NaN,None,A0425,NaN,None,None,None,None,None


#### Dont know what to do with the columns value_as_string, value_as_concept_id

In [55]:
df = df.drop(['observation_datetime',
              'value_as_number',
              'qualifier_concept_id',
              'visit_occurrence_id',
              'visit_detail_id',
              'observation_source_value',
              'observation_source_concept_id',
              'observation_event_id',
              'unit_source_value',
              'qualifier_source_value',
              'observation_event_id	',
              'obs_event_field_concept_id',
              'value_as_datetime',
              'unit_concept_id',
              'provider_id'], axis=1, errors='ignore')

In [56]:
df['observation_concept_id'] = df['observation_concept_id'].apply(str)
df['observation_type_concept_id'] = df['observation_type_concept_id'].apply(str)

In [57]:
def agg_observation_concept_id(x):
    return pd.Series(dict(
        observation_concept_id_list  = ', '.join(set(x.observation_concept_id)),
        observation_type_concept_id_list  = ', '.join(set(x.observation_type_concept_id))
        ))

In [58]:
agg_dict = {'person_id': 'max',
            'observation_date': 'min'}

rename_dict = {'observation_date': 'days_since_latest_observation'}

group_by_var = 'person_id'
date_var = 'observation_date'
apply_func = agg_observation_concept_id

df.observation_date = pd.to_datetime(df.observation_date, format='%Y-%m-%d')
observation_data = \
window_data(df, window_size, window_start, group_by_var, date_var, agg_dict, rename_dict, apply_func)

In [59]:
training_data = pd.merge(training_data, observation_data, on=['person_id', 'window_id'], how='left')
training_data.head()

,person_id,days_since_latest_visit,ethnicity_concept_id,race_concept_id,gender_concept_id,race_concept_name,number_of_visits,visit_concept_name,visit_duration,death_in_next_window,...,days_since_latest_procedure,procedure_concept_id_list,procedure_type_concept_id_list,days_since_latest_drug_exposure,total_quantity_of_drugs,drug_concept_id_list,drug_type_concept_id_list,days_since_latest_observation,observation_concept_id_list,observation_type_concept_id_list
0,0,4 days,NaN,8552.0,8507,Unknown,5,1,0 days,0,...,62 days,"2414392, 4048727, 4306780",38000269,NaT,NaN,NaN,NaN,8 days,"4214956, 439405, 440927, 0",38000282
1,6,142 days,NaN,8527.0,8532,White,3,0,0 days,0,...,NaT,NaN,NaN,NaT,NaN,NaN,NaN,63 days,"4149299, 440927, 4015724, 2514491, 439405",38000282
2,10,123 days,38003564.0,8552.0,8532,Unknown,1,0,0 days,0,...,32 days,"2108115, 2211338, 2001200, 0",38000269,33 days,110.0,"798875, 19133566",38000175,NaT,NaN,NaN
3,13,10 days,38003564.0,8552.0,8532,Unknown,5,2,0 days,0,...,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN
4,16,81 days,NaN,8516.0,8507,Black or African American,1,0,0 days,0,...,2 days,"2314285, 2514408, 2002291",38000269,39 days,60.0,"19102504, 40006185, 1309069",38000175,NaT,NaN,NaN


In [60]:
training_data.columns

Index(['person_id', 'days_since_latest_visit', 'ethnicity_concept_id',
       'race_concept_id', 'gender_concept_id', 'race_concept_name',
       'number_of_visits', 'visit_concept_name', 'visit_duration',
       'death_in_next_window', 'old', 'inpatient_visit_count',
       'outpatient_visit_count', 'er_visit_count', 'window_id',
       'days_since_latest_condition', 'condition_status_concept_id',
       'condition_concept_id_list', 'condition_type_concept_id_list',
       'days_since_latest_procedure', 'procedure_concept_id_list',
       'procedure_type_concept_id_list', 'days_since_latest_drug_exposure',
       'total_quantity_of_drugs', 'drug_concept_id_list',
       'drug_type_concept_id_list', 'days_since_latest_observation',
       'observation_concept_id_list', 'observation_type_concept_id_list'],
      dtype='object')

In [61]:
del observation_data